## Probability grids

This notebook creates the time-dependent probability maps and writes them to file (`.nc` format). The notebook `01a-create_classifiers_global.ipynb` must have been run previously.

In [1]:
use_extracted_data = True


In [2]:
import os
import time
from datetime import timedelta

import dask
import dask.dataframe as dd
from dask_ml.wrappers import ParallelPostFit
from joblib import load

from lib.check_files import check_prepared_data
from lib.pu import create_probability_grids


In [3]:
n_jobs = int(os.environ.get("N_JOBS", 8))
dask.config.set(num_workers=n_jobs)


### Load input data from file

In [4]:
if use_extracted_data:
    data_dir = "extracted_data"
else:
    data_dir = "prepared_data"
    check_prepared_data(data_dir, verbose=True)
data_filename = os.path.join(data_dir, "grid_data.csv")
point_data = dd.read_csv(data_filename)

df_out = point_data[["lon", "lat", "age (Ma)"]].compute()


### Calculate probabilities

In [5]:
output_dir = os.path.join("outputs", "global")

for algorithm in ("PU", "SVM"):
    print(
        f"Calculating probabilities for {algorithm} model... ",
        end="",
        flush=True,
    )
    t0 = time.time()

    subdir = os.path.join(output_dir, algorithm)
    model_filename = os.path.join(subdir, f"classifier.joblib")
    probabilities_filename = os.path.join(
        subdir,
        f"grid_probabilities.csv",
    )
    model = load(model_filename)

    # Set model n_jobs if possible
    # (let dask handle parallelism at this stage)
    try:
        model[-1].set_params(n_jobs=1)
    except ValueError:
        pass

    model_parallel = ParallelPostFit(model)

    point_x = point_data[model.feature_names_in_]
    p = model_parallel.predict_proba(point_x)[:, 1].ravel().compute()
    probabilities = df_out.copy()
    probabilities["probability"] = p
    del p
    probabilities.to_csv(probabilities_filename, index=False)
    del probabilities, model
    duration = timedelta(seconds=time.time() - t0)
    print(f"Done! (duration: {duration})", flush=True)

del point_data


Calculating probabilities for PU model... Done! (duration: 0:01:31.628367)
Calculating probabilities for SVM model... Done! (duration: 0:00:22.702945)


### Create probability maps

In [6]:
for algorithm in ("PU", "SVM"):
    print(
        f"Creating grids for {algorithm} model... ",
        end="",
        flush=True,
    )
    t0 = time.time()

    subdir = os.path.join(output_dir, algorithm)
    probabilities_filename = os.path.join(
        subdir,
        f"grid_probabilities.csv",
    )
    grid_output_dir = os.path.join(
        subdir,
        f"probability_grids",
    )
    os.makedirs(grid_output_dir, exist_ok=True)

    create_probability_grids(
        data=probabilities_filename,
        output_dir=grid_output_dir,
        threads=n_jobs,
        extent=(-180, 180, -90, 90),
    )
    duration = timedelta(seconds=time.time() - t0)
    print(f"Done! (duration: {duration})", flush=True)


Creating grids for PU model... Done! (duration: 0:01:34.552722)
Creating grids for SVM model... Done! (duration: 0:01:30.782732)


In [7]:
import xarray as xr
import dask.array as da
import numpy as np
import pandas as pd

# Define dimensions
lat = 1801
lon = 3601
time_steps = 2000  # Number of time steps

# Create coordinates for latitude, longitude, and time
latitudes = np.linspace(-90, 90, lat)
longitudes = np.linspace(-180, 180, lon)
time = np.arange(1, time_steps + 1)  # Time steps from 1 to 2000

# Create empty Dask arrays for data variables
band1 = da.zeros((time_steps, lat, lon), dtype='float32', chunks=(10, 100, 100))  # Adjust chunk sizes as needed

# Create the dataset
ds = xr.Dataset(
    {
        'Band1': (('time', 'lat', 'lon'), band1),
    },
    coords={
        'lat': ('lat', latitudes),
        'lon': ('lon', longitudes),
        'time': ('time', time),
    }
)

# Add attributes
ds.attrs['Conventions'] = 'CF-1.5'
ds.attrs['GDAL'] = 'GDAL 3.3.0, released 2021/04/26'

# To check the size without computing the data
print(ds)

# Get the estimated size in bytes
size_in_bytes = ds.nbytes  # This will give you the size of the dataset in bytes
print(f"Estimated size: {size_in_bytes / (1024 * 1024 * 1024):.2f} GB")

<xarray.Dataset> Size: 52GB
Dimensions:  (time: 2000, lat: 1801, lon: 3601)
Coordinates:
  * lat      (lat) float64 14kB -90.0 -89.9 -89.8 -89.7 ... 89.7 89.8 89.9 90.0
  * lon      (lon) float64 29kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * time     (time) int64 16kB 1 2 3 4 5 6 7 ... 1995 1996 1997 1998 1999 2000
Data variables:
    Band1    (time, lat, lon) float32 52GB dask.array<chunksize=(10, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 3.3.0, released 2021/04/26
Estimated size: 48.32 GB


In [3]:
ds

<xarray.Dataset> Size: 52GB
Dimensions:  (time: 2000, lat: 1801, lon: 3601)
Coordinates:
  * lat      (lat) float64 14kB -90.0 -89.9 -89.8 -89.7 ... 89.7 89.8 89.9 90.0
  * lon      (lon) float64 29kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * time     (time) int64 16kB 1 2 3 4 5 6 7 ... 1995 1996 1997 1998 1999 2000
Data variables:
    Band1    (time, lat, lon) float32 52GB dask.array<chunksize=(10, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.5
    GDAL:         GDAL 3.3.0, released 2021/04/26

In [5]:
ds.nbytes/1e9


51.883267216

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\03_Data_Science_Pipeline\\MGE_Phase_4_Global_Standards\\MGE_Phase_4_PCu_Age_Bins_20240905.csv'